# Creating Pandapower Networks

This Tutorial will introduce the user into the pandapower datastructure and how to create networks through the pandapower API. The following example contains all basic elements that are supported by the Pandapower format.


![](http://a-versys.iwes.fraunhofer.de/doc-devel/html/_images/example_network_simple.png)




<img src="pics/example_network_simple.png">

## Creating network components

The datastructure of the pandapower framework is based on the python library pandas. A pandapower network consist of a separate element table for each element type that is used in the network. Each element table consists of a column for each parameter and a row for each element. By executing the follwing code cells you generate the various element tables.  A detailed description about each parameter you will find [here] [identification tag for link_2].

[identification tag for link_2]: http://a-versys.iwes.fraunhofer.de/doc-devel/html/pandapower/datastructure.html

In [76]:
import pandapower as pp #import pandapower

net = pp.create_empty_network() #create an empty network

### Buses

First, we create the three high voltage (vn_kv=110.) and six medium voltage (vn_kv=20.) buses.

<img src="pics/example_network_simple_buses.png">

In [77]:
# Bus 3 and bus 4 are classified as nodes. The type parameter is set to type="n". 
# All other bus types are declared as busbars: (type="b")
bus1 = pp.create_bus(net, name="HV Busbar", vn_kv=110, type="b")
bus2 = pp.create_bus(net, name="HV Busbar 2", vn_kv=110, type="b")
bus3 = pp.create_bus(net, name="HV Transformer Bus", vn_kv=110, type="n")
bus4 = pp.create_bus(net, name="MV Transformer Bus", vn_kv=20, type="n")
bus5 = pp.create_bus(net, name="MV Station 1", vn_kv=20, type="b")
bus6 = pp.create_bus(net, name="MV Station 2", vn_kv=20, type="b")
bus7 = pp.create_bus(net, name="MV Station 3", vn_kv=20, type="b")
bus8 = pp.create_bus(net, name="MV Station 4", vn_kv=20, type="b")

net.bus # display bus table after creation of bus elements

,name,vn_kv,type,zone,in_service
0,HV Busbar,110.0,b,None,True
1,HV Busbar 2,110.0,b,None,True
2,HV Transformer Bus,110.0,n,None,True
3,MV Transformer Bus,20.0,n,None,True
4,MV Station 1,20.0,b,None,True
5,MV Station 2,20.0,b,None,True
6,MV Station 3,20.0,b,None,True
7,MV Station 4,20.0,b,None,True


### External network

We now create an external grid connection that serves as slack node for the power flow calculation. The voltage of the external grid is set to a magnitude of 1.0 per unit and 20 degrees voltage angle.

<img src="pics/example_network_simple_ext_grids.png">

In [78]:
pp.create_ext_grid(net, bus1, vm_pu=1.01, va_degree=50) # Create an external grid connection

net.ext_grid #show the external grid table

,name,bus,vm_pu,va_degree,in_service
0,None,0,1.01,50.0,True


### Transformer 

The transformer connects the high-voltage with the medium-voltage side of the grid. The high-voltage bus of the transformer is connected to Bus3 and on the medium-voltage side the transformer is linked to Bus4. In this case we select a standard type transformer from the pandapower standard type library ("110kV/20kV transformer"). The detailled transformer parameters, such as short circuit voltages, rated power or iron losses are automatically loaded from the standard type library.


<img src="pics/example_network_simple_trafos.png">

In [79]:
# We create the transformer out of the Pandapower standard type library:
pp.create_transformer(net, bus3, bus4, name="110kV/20kV transformer", std_type="25 MVA 110/20 kV")

# detailed transformer parameters:
net.trafo

,name,std_type,hv_bus,lv_bus,sn_kva,vn_hv_kv,vn_lv_kv,vsc_percent,vscr_percent,pfe_kw,i0_percent,tp_side,tp_mid,tp_min,tp_max,tp_st_percent,tp_pos,in_service,shift_degree
0,110kV/20kV transformer,25 MVA 110/20 kV,2,3,25000.0,110.0,20.0,11.2,0.282,29.0,0.071,hv,0,-9,9,1.5,0,True,150.0


### Lines

The network includes three medium voltage lines and one high voltage line. The bus connections are implemented as shown in the network diagram:

<img src="pics/example_network_simple_lines.png">

The line parameters are once again taken from the standard type library. The full line table looks like this:

In [80]:
line1=pp.create_line(net, bus1, bus2, length_km=10, std_type="N2XS(FL)2Y 1x300 RM/35 64/110 kV", name="Line 1")
line2=pp.create_line(net, bus5, bus6, length_km=2, std_type="NA2XS2Y 1x240 RM/25 12/20 kV", name="Line 2")
line3=pp.create_line(net, bus5, bus7, length_km=3.5, std_type="48-AL1/8-ST1A 20.0", name="Line 3")
line4=pp.create_line(net, bus5, bus8, length_km=2.5, std_type="NA2XS2Y 1x240 RM/25 12/20 kV", name="Line 4")

In [81]:
net.line # line table

,name,std_type,from_bus,to_bus,length_km,r_ohm_per_km,x_ohm_per_km,c_nf_per_km,imax_ka,df,parallel,type,in_service
0,Line 1,N2XS(FL)2Y 1x300 RM/35 64/110 kV,0,1,10.0,0.0600,0.144,144.0,0.588,1.0,1,cs,True
1,Line 2,NA2XS2Y 1x240 RM/25 12/20 kV,4,5,2.0,0.1220,0.112,304.0,0.421,1.0,1,cs,True
2,Line 3,48-AL1/8-ST1A 20.0,4,6,3.5,0.5939,0.372,9.5,0.210,1.0,1,ol,True
3,Line 4,NA2XS2Y 1x240 RM/25 12/20 kV,4,7,2.5,0.1220,0.112,304.0,0.421,1.0,1,cs,True


### Switches

The switches within the grid can be assigned to two different groups of element types. The circuit breakers on the high- and low voltage side of the transformer are located between
the two bus pairs: (Bus2/Bus3 and Bus4/Bus5). These switches represent bus-bus switches (et="b"). The remaining load break switches are assigned as line-bus switches (et="l"). Switches with this
element type connect one line with one bus.

<img src="pics/example_network_simple_switches.png">

In [82]:
# We create bus-bus switches (et="b"):
pp.create_switch(net, bus2, bus3, et="b", type="CB")
pp.create_switch(net, bus4, bus5, et="b", type="CB")
# We create line-bus switches (et="l"):
pp.create_switch(net, bus5, line2, et="l", type="LBS")
pp.create_switch(net, bus6, line2, et="l", type="LBS")
pp.create_switch(net, bus5, line3, et="l", type="LBS")
pp.create_switch(net, bus7, line3, et="l", type="LBS")
pp.create_switch(net, bus5, line4, et="l", type="LBS")
pp.create_switch(net, bus8, line4, et="l", type="LBS")

# switch table:
net.switch

,bus,element,et,type,closed,name
0,1,2,b,CB,True,None
1,3,4,b,CB,True,None
2,4,1,l,LBS,True,None
3,5,1,l,LBS,True,None
4,4,2,l,LBS,True,None
5,6,2,l,LBS,True,None
6,4,3,l,LBS,True,None
7,7,3,l,LBS,True,None


### Generator / static generator and load

The next step is the creation of the generator, the static generator and the load. We connect the components to our busbars at the medium-voltage side and declare the different parameters within each element. 


<img src="pics/example_network_simple_gens_sgens_loads.png">

In [83]:
pp.create_gen(net, bus6, p_kw=-6000, vm_pu=1.03, name="generator") 

net.gen

,name,bus,p_kw,vm_pu,sn_kva,min_q_kvar,max_q_kvar,scaling,in_service,type
0,generator,5,-6000.0,1.03,NaN,NaN,NaN,1.0,True,None


In [84]:
pp.create_sgen(net, bus7, p_kw=-2000, name="static generator")

net.sgen

,name,bus,p_kw,q_kvar,sn_kva,scaling,in_service,type,controllable
0,static generator,6,-2000.0,0.0,NaN,1.0,True,None,False


In [85]:
pp.create_load(net, bus8, p_kw=2000, q_kvar=4000, scaling=0.6, name="load")

net.load

,name,bus,p_kw,q_kvar,sn_kva,scaling,in_service,type
0,load,7,2000.0,4000.0,NaN,0.6,True,None


### Shunt

The last step is the creation of the shunt. We connect the shunt to bus3 and declare the different shunt properties

<img src="pics/example_network_simple_shunts.png">

In [86]:
pp.create_shunt(net, bus3, p_kw=0, q_kvar=-960, name='Shunt')

net.shunt

,bus,name,p_kw,q_kvar,in_service
0,2,Shunt,0.0,-960.0,1


## Run loadflow and generate result tables

By executing a loadflow, the following pandapower tables contain all the results of the different components (bus, line, trafo,  ext_grid, gen, sgen, load):

In [87]:
# Execution of the loadflow:
pp.runpp(net)

# Result tables generated:
net

This pandapower network includes the following parameter tables:
   - bus (8 elements)
   - line (4 elements)
   - ext_grid (1 elements)
   - load (1 elements)
   - shunt (1 elements)
   - sgen (1 elements)
   - switch (8 elements)
   - trafo (1 elements)
   - gen (1 elements)
 and the following results tables:
   - res_gen (1 elements)
   - res_bus (8 elements)
   - res_ext_grid (1 elements)
   - res_sgen (1 elements)
   - res_trafo (1 elements)
   - res_load (1 elements)
   - res_shunt (1 elements)
   - res_line (4 elements)

You can easily get access to the different result tables by executing:

In [90]:
# generate line result table:
pp.runpp(net, calculate_voltage_angles=True, init="dc")
net.res_line

,p_from_kw,q_from_kvar,p_to_kw,q_to_kvar,pl_kw,ql_kvar,i_ka,loading_percent
0,-6699.876767,-9139.137559,6704.017095,3.559181e+03,4.140328,-5579.957038,0.058888,10.015003
1,-5964.286352,-5117.281812,6000.000000,5.069508e+03,35.713648,-47.773467,0.221616,52.640394
2,-1980.547110,7.763060,2000.000000,1.267586e-08,19.452890,7.763060,0.055852,26.596298
3,1205.091866,2304.840564,-1200.000000,-2.400000e+03,5.091866,-95.159436,0.075855,18.017701


In [92]:
net.res_trafo

,p_hv_kw,q_hv_kvar,p_lv_kw,q_lv_kvar,pl_kw,ql_kvar,i_hv_ka,i_lv_ka,loading_percent
0,-6704.017095,-2577.795014,6739.741596,2804.678189,35.724501,226.883174,0.037286,0.205861,28.524953


In [93]:
net.res_shunt

,p_kw,q_kvar,vm_pu
0,0.0,-981.385507,1.011077


In [94]:
net.res_bus

,vm_pu,va_degree,p_kw,q_kvar
0,1.010000,0.000000,6699.876767,9139.137506
1,1.011077,0.027078,0.000000,0.000000
2,1.011077,0.027078,0.000000,-981.385507
3,1.023668,1.676997,0.000000,0.000000
4,1.023668,1.676997,0.000000,0.000000
5,1.030000,1.690196,-6000.000000,-5069.508314
6,1.033710,2.028860,-2000.000000,0.000000
7,1.021161,1.729179,1200.000000,2400.000000
